In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Python code in one [module](https://docs.python.org/3/glossary.html#term-module) gains access to the code in another module by the process of [importing](https://docs.python.org/3/glossary.html#term-importing) it. The [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) statement is the most common way of invoking the import machinery, but it is not the only way. Functions such as [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) and built-in [`__import__()`](https://docs.python.org/3/library/functions.html#import__) can also be used to invoke the import machinery.

The [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) statement combines two operations; it searches for the named module, then it binds the results of that search to a name in the local scope. The search operation of the `import` statement is defined as a call to the [`__import__()`](https://docs.python.org/3/library/functions.html#import__) function, with the appropriate arguments. The return value of [`__import__()`](https://docs.python.org/3/library/functions.html#import__) is used to perform the name binding operation of the `import` statement. See the `import` statement for the exact details of that name binding operation.

A direct call to [`__import__()`](https://docs.python.org/3/library/functions.html#import__) performs only the module search and, if found, the module creation operation. While certain side-effects may occur, such as the importing of parent packages, and the updating of various caches (including [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules)), only the [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) statement performs a name binding operation.

When an [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) statement is executed, the standard builtin [`__import__()`](https://docs.python.org/3/library/functions.html#import__) function is called. Other mechanisms for invoking the import system (such as [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module)) may choose to bypass [`__import__()`](https://docs.python.org/3/library/functions.html#import__) and use their own solutions to implement import semantics.

When a module is first imported, Python searches for the module and if found, it creates a module object [1](https://docs.python.org/3/reference/import.html#fnmo), initializing it. If the named module cannot be found, a [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) is raised. Python implements various strategies to search for the named module when the import machinery is invoked. These strategies can be modified and extended by using various hooks described in the sections below.

*Changed in version 3.3:* The import system has been updated to fully implement the second phase of [**PEP 302**](https://www.python.org/dev/peps/pep-0302). There is no longer any implicit import machinery - the full import system is exposed through [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path). In addition, native namespace package support has been implemented (see [**PEP 420**](https://www.python.org/dev/peps/pep-0420)).

> 一个 [模块](https://docs.python.org/3/glossary.html#term-module) 中的 Python 代码通过 [importing](https://docs.python.org/3/glossary.html#term-importing) 过程获得对另一个模块中的代码的访问。[`import`](https://docs.python.org/3/reference/simple_stmts.html#import) 语句是调用导入机制的最常见方式，但它不是唯一的方式。诸如 [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) 和内置的 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 等函数也可以用来调用导入机制。
>
> [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) 语句结合了两个操作；它搜索指定的模块，然后将搜索结果绑定到局部作用域的一个名字。`import`语句的搜索操作被定义为对 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 函数的调用，并带有适当的参数。[`__import__()`](https://docs.python.org/3/library/functions.html#import__) 的返回值被用来执行 `import` 语句的名称绑定操作。关于该名称绑定操作的具体细节，请参见`import`语句。
>
> 直接调用 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 只执行模块搜索，如果找到，则执行模块创建操作。虽然可能会出现某些副作用，例如导入父包，以及更新各种缓存（包括 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules)），但只有 [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) 语句会执行名称绑定操作。
>
> 当[`import`](https://docs.python.org/3/reference/simple_stmts.html#import)语句被执行时，标准的内置 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 函数被调用。其他调用导入系统的机制 (例如 [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module)) 可以选择绕过 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 并使用自己的方案来实现导入语义。
>
> 当一个模块第一次被导入时，Python 会搜索这个模块，如果找到了，它会创建一个模块对象 [^1]，并初始化它。如果找不到指定的模块，就会产生 [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) 。当调用导入机制时，Python 实现了各种策略来搜索指定的模块。这些策略可以通过使用下面几节中描述的各种钩子进行修改和扩展。
>
> *3.3 版中的变化：* 导入系统已经更新，以完全实现 [**PEP 302**](https://www.python.org/dev/peps/pep-0302) 的第二阶段。不再有任何隐含的导入机制--完整的导入系统通过[`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path)公开。此外，本地名称空间包支持已经实现（见[**PEP 420**](https://www.python.org/dev/peps/pep-0420)）。

## 5.1. [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib)

The [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib) module provides a rich API for interacting with the import system. For example [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) provides a recommended, simpler API than built-in [`__import__()`](https://docs.python.org/3/library/functions.html#import__) for invoking the import machinery. Refer to the [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib) library documentation for additional detail.

> [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib) 模块提供了丰富的API来与导入系统进行交互。例如 [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) 提供了一个推荐的、比内置的 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 更简单的 API 来调用导入机制。请参考 [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib) 库文档以了解更多细节。

## 5.2. Packages

Python has only one type of module object, and all modules are of this type, regardless of whether the module is implemented in Python, C, or something else. To help organize modules and provide a naming hierarchy, Python has a concept of [packages](https://docs.python.org/3/glossary.html#term-package).

You can think of packages as the directories on a file system and modules as files within directories, but don’t take this analogy too literally since packages and modules need not originate from the file system. For the purposes of this documentation, we’ll use this convenient analogy of directories and files. Like file system directories, packages are organized hierarchically, and packages may themselves contain subpackages, as well as regular modules.

It’s important to keep in mind that all packages are modules, but not all modules are packages. Or put another way, packages are just a special kind of module. Specifically, any module that contains a `__path__` attribute is considered a package.

All modules have a name. Subpackage names are separated from their parent package name by a dot, akin to Python’s standard attribute access syntax. Thus you might have a package called [`email`](https://docs.python.org/3/library/email.html#module-email), which in turn has a subpackage called [`email.mime`](https://docs.python.org/3/library/email.mime.html#module-email.mime) and a module within that subpackage called `email.mime.text`.

> Python 只有一种类型的模块对象，所有的模块都是这种类型的，不管这个模块是用 Python、C 还是其它语言实现的。为了帮助组织模块并提供一个命名层次，Python 有一个 [包（packages）](https://docs.python.org/3/glossary.html#term-package) 的概念。
>
> 你可以把包想象成文件系统中的目录，把模块想象成目录中的文件，但不要把这个比喻看得太重，因为包和模块不一定来自文件系统。在本文档中，我们将使用这个方便的目录和文件的类比。就像文件系统的目录一样，包是分层次组织的，包本身可能包含子包，以及常规模块。
>
> 重要的是要记住，所有的包都是模块，但不是所有的模块都是包。或者换句话说，包只是模块的一种特殊类型。具体来说，任何包含 `__path__` 属性的模块都被认为是一个包。
>
> 所有模块都有一个名字。子包名与它们的父包名用一个点隔开，类似于 Python 的标准属性访问语法。因此你可能有一个名为 [`email`](https://docs.python.org/3/library/email.html#module-email) 的包，它又有一个名为 [`email.mime`](https://docs.python.org/3/library/email.mime.html#module-email.mime) 的子包，以及该子包中的一个模块，名为 `email.mime.text`。

### 5.2.1. Regular packages

Python defines two types of packages, [regular packages](https://docs.python.org/3/glossary.html#term-regular-package) and [namespace packages](https://docs.python.org/3/glossary.html#term-namespace-package). Regular packages are traditional packages as they existed in Python 3.2 and earlier. A regular package is typically implemented as a directory containing an `__init__.py` file. When a regular package is imported, this `__init__.py` file is implicitly executed, and the objects it defines are bound to names in the package’s namespace. The `__init__.py` file can contain the same Python code that any other module can contain, and Python will add some additional attributes to the module when it is imported.

For example, the following file system layout defines a top level `parent` package with three subpackages:

> Python 定义了两种类型的包，[常规包（regular packages）](https://docs.python.org/3/glossary.html#term-regular-package) 和 [名称空间包（namespace packages）](https://docs.python.org/3/glossary.html#term-namespace-package) 。常规包是传统的包，因为它们存在于 Python 3.2 和更早的版本中。一个常规包通常被实现为一个包含 `__init__.py` 文件的目录。当一个常规包被导入时，这个 `__init__.py` 文件被隐式执行，它所定义的对象被绑定到包的名称空间中。`__init__.py`文件可以包含和其他模块一样的 Python 代码，当模块被导入时，Python 会给它添加一些额外的属性。
>
> 例如，下面的文件系统布局定义了一个带有三个子包的顶层 `parent` 包：

Importing `parent.one` will implicitly execute `parent/__init__.py` and `parent/one/__init__.py`. Subsequent imports of `parent.two` or `parent.three` will execute `parent/two/__init__.py` and `parent/three/__init__.py` respectively.

> 导入 `parent.one` 将隐含地执行 `parent/__init__.py` 和 `parent/one/__init__.py`。随后导入 `parent.two` 或 `parent.three` 将分别执行 `parent/two/__init__.py` 和 `parent/three/__init__.py`。